In [1]:
#!pip install ipynb --upgrade

In [2]:
#!pip install --upgrade pip

In [3]:
#!pip install keras-tcn
#!pip install tensorflow
#!pip install ipynb
#!pip install plotly

In [4]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
sys.path.append('C:/Users/Alex-/Desktop/Beslutsstödssystem/models-decision-system')
os.chdir("..")
import Decision
import Models

os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
os.chdir("../models-decision-system/MockPipeline")

In [5]:
import os

In [6]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [7]:
settings = load_yaml('mock_settings.yaml')

In [8]:
dataframe = processing.create_dataframe(settings)

dataset = features.add(dataframe, settings['features'])

dataset

data_linreg = features.split(dataset, 'linreg', settings['ensemble']['models'][0]['linreg'])

linreg = Models.train_model(data_linreg, 'linreg', settings['ensemble']['models'][0])

data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

settings['ensemble']['models'][1]['lstm']['layers']

lstm = Models.train_model(data_lstm, 'lstm', settings['ensemble']['models'][1]['lstm'])

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
from sklearn import preprocessing
from sklearn import utils

In [9]:
df = pd.read_csv("../MockData.csv")
df_dec = pd.read_csv("../decision.csv")

In [10]:
df_dec = df_dec.set_index("DateTime")
df = df.set_index("DateTime")

In [11]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split

In [12]:
df

,Close,RoC,S%D,PSY12,ASY5
DateTime,,,,,
2010-01-27,0.817467,0.279074,0.239967,0.272727,0.452398
2010-01-28,0.802634,0.256945,0.173515,0.272727,0.376690
2010-01-29,0.779823,0.260848,0.136936,0.272727,0.291014
2010-01-31,0.782677,0.276472,0.108876,0.363636,0.304610
2010-02-01,0.793195,0.281944,0.078999,0.363636,0.385942
...,...,...,...,...,...
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453
2019-12-27,0.175454,0.565530,0.421044,0.818182,0.586624
2019-12-29,0.177701,0.567630,0.451166,0.818182,0.585169


In [13]:
df_dec

,Forecast,Actual,f_momentum,a_momentum,Invest next day
DateTime,,,,,
2018-05-22,0.330828,0.296319,NaN,NaN,Hold
2018-05-23,0.333239,0.299195,NaN,NaN,Hold
2018-05-24,0.328026,0.282542,-0.002801,-0.013777,Sell
2018-05-25,0.322993,0.291667,-0.010246,-0.007529,Hold
2018-05-27,0.310946,0.277665,-0.017081,-0.004877,Buy
...,...,...,...,...,...
2019-12-24,0.168590,0.158172,-0.001180,0.000607,Hold
2019-12-25,0.167789,0.159700,-0.000904,0.003124,Hold
2019-12-26,0.166317,0.175454,-0.002273,0.017282,Hold


In [14]:
col = "Invest next day"
conditions = [df_dec[col] == "Buy"]
choices = ["1"]
df_dec["buy"] = np.select(conditions, choices, default="0")
conditions = [df_dec[col] == "Sell"]
choices = ["1"]
df_dec["sell"] = np.select(conditions, choices, default="0")
conditions = [df_dec[col] == "Hold"]
choices = ["1"]
df_dec["hold"] = np.select(conditions, choices, default="0")

In [15]:
df_dec

,Forecast,Actual,f_momentum,a_momentum,Invest next day,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.330828,0.296319,NaN,NaN,Hold,0,0,1
2018-05-23,0.333239,0.299195,NaN,NaN,Hold,0,0,1
2018-05-24,0.328026,0.282542,-0.002801,-0.013777,Sell,0,1,0
2018-05-25,0.322993,0.291667,-0.010246,-0.007529,Hold,0,0,1
2018-05-27,0.310946,0.277665,-0.017081,-0.004877,Buy,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.168590,0.158172,-0.001180,0.000607,Hold,0,0,1
2019-12-25,0.167789,0.159700,-0.000904,0.003124,Hold,0,0,1
2019-12-26,0.166317,0.175454,-0.002273,0.017282,Hold,0,0,1


In [16]:
df["buy"] = df_dec["buy"]
df["sell"] = df_dec["sell"]
df["hold"] = df_dec["hold"]

In [17]:
df = df.dropna()
df

,Close,RoC,S%D,PSY12,ASY5,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.314006,0.480472,0.194738,0.363636,0.479935,0,0,1
2018-05-23,0.296319,0.444085,0.198627,0.363636,0.432865,0,0,1
2018-05-24,0.299195,0.411007,0.185304,0.363636,0.462282,0,1,0
2018-05-25,0.282542,0.349541,0.151715,0.272727,0.400889,0,0,1
2018-05-27,0.291667,0.366831,0.125065,0.272727,0.414903,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.156576,0.526441,0.519878,0.727273,0.477280,0,0,1
2019-12-25,0.158172,0.504520,0.453905,0.727273,0.476746,0,0,1
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453,0,0,1


In [18]:
df.columns[:5]

Index(['Close', 'RoC', 'S%D', 'PSY12', 'ASY5'], dtype='object')

In [19]:
df

,Close,RoC,S%D,PSY12,ASY5,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.314006,0.480472,0.194738,0.363636,0.479935,0,0,1
2018-05-23,0.296319,0.444085,0.198627,0.363636,0.432865,0,0,1
2018-05-24,0.299195,0.411007,0.185304,0.363636,0.462282,0,1,0
2018-05-25,0.282542,0.349541,0.151715,0.272727,0.400889,0,0,1
2018-05-27,0.291667,0.366831,0.125065,0.272727,0.414903,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.156576,0.526441,0.519878,0.727273,0.477280,0,0,1
2019-12-25,0.158172,0.504520,0.453905,0.727273,0.476746,0,0,1
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453,0,0,1


In [20]:
def foofoofoo(lblz):
    container = []
    for row in lblz:
        word = ''
        for value in row:
            word = word + str(value)
        container.append(word)
    return container

In [21]:
labels = df[["buy", "sell", "hold"]].to_numpy()

In [22]:
labels = foofoofoo(labels)

In [23]:
forecast_days = 1
#df['Label'] = df['Close'].shift(-forecast_days)
df_dec.dropna()
data = df
data = data.dropna(how="any")
x = data[df.columns[:5]].to_numpy()
y = data[df.columns[-4]].to_numpy()
y2 = data[df.columns[-3:]].to_numpy()

## Dataset y


In [24]:
def create_split(x, y):
    tss = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tss.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
    return x_train, y_train, x_test, y_test

In [25]:
x_train, y_train, x_test, y_test = create_split(x, y)
x_train_2, y_train_2, x_test_2, y_test_2 = create_split(x, y2)

In [26]:
y_test_2 = foofoofoo(y_test_2)
y_train_2 = foofoofoo(y_train_2)

## Dataset y2

In [27]:
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import LinearSVC
#OneVsRestClassifier(LinearSVC(random_state=0)).fit(x, y).predict(x)

In [28]:
from sklearn.preprocessing import MinMaxScaler

In [29]:
scaler = MinMaxScaler(feature_range=(0,1))

x_train = scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

In [30]:
x = scaler.transform(x)

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y_train)
print(training_scores_encoded)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(training_scores_encoded))

In [31]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from pprint import pprint
from sklearn import metrics

# Gridsearch random forest

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74


### Functions

In [32]:
def random_search(settings):
    
    # CREATE THE CORRECT MODEL TYPE
    model = select_model(settings)

    # CREATE THE RANDOMIZED SEARCH CV
    rscv = RandomizedSearchCV(estimator=model,
                                  param_distributions=settings['random_params'],
                                  n_iter=100,
                                  cv = 3,
                                  verbose=2,
                                  random_state=42,
                                  n_jobs=-1)

    # FIT THE RANDOM SEARCH
    rscv.fit(settings["x"], settings["y"])
    
    # SELECT THE BEST ESTIMATOR
    best_random = rscv.best_estimator_

    
    
    return rscvm, rscv.best_params_

In [33]:
def grid_search(model, dataset, settings):

    
    # SELECT MODEL
    model = select_model(settings)

    # CREATE GRID SEARCH CV
    grid_search = GridSearchCV(estimator = model, param_grid = settings["grid_params"], 
                          cv = settings["cv"], n_jobs = settings["n_jobs"], verbose = 2)
    grid_search.fit(dataset["x"], dataset["y"])
    best_grid = grid_search.best_estimator_

    return best_grid

In [34]:
def select_model(settings):
    """Select a classification model type. 
    Supported models:
    Random Forest Classifier (default)"""
    if(settings["model"] == "rfc"):
        model = RandomForestClassifier()
    else:
        model = RandomForestClassifier()
    return model

In [35]:
def grid_search_model(model, dataset, settings):
    """Runs a grid search on the given model to define the best parameters for the model. 
    
    Parameters:
    -----------
    model: sklearn model 
    unfitted model
    dataset: dict
    Dictionary that contains x_train, y_train, features and labels
    
    Returns:
    --------
    model: sklearn model
    Same type of model as the input model, with optimized parameters
    """
    # CREATE GRID SEARCH CV
    grid_search = GridSearchCV(estimator = model, param_grid = settings["grid_params"], 
                          cv = settings["cv"], n_jobs = settings["n_jobs"], verbose = 2)
    
    # FIT GRIDSEARCH
    grid_search.fit(dataset["features"], dataset["labels"])
    
    # PICK OUT THE BEST MODEL FROM GRID SEARCH
    model = grid_search.best_estimator_
    
    # RUN MODEL ON BEST PARAMS   
    model.fit(dataset["x_train"], dataset["y_train"])
    
    # RETURN MODEL WITH BEST PARAMS    
    return model

In [36]:
from sklearn import preprocessing
def evaluate(model, test_features, test_labels, encode=True):
    """Evaluate a classifcation model, for example Random Forest Classification
    
    Parameters:
    model: sklearn classification model
    test_features: np.array of features
    test_labels: np.array of labels 
    encode (default=True): boolean, encodes values to int if set to true
    
    Returns:
    accuracy: float, metrics.accuracy_score
    precision: float, metrics.precision_score
    recall: float, metrics.precision_score
    confusion: 
    """
    # PREDICT
    predict = model.predict(test_features)
    predict_proba = model.predict_proba(test_features)

    # OPTIONAL ENCODING
    if(encode == True):
        le = preprocessing.LabelEncoder()
        le.fit(test_labels)
        test_labels = le.transform(test_labels)
        predictions = le.transform(predict)
        
    #binary_labels = preprocessing.binarize(test_labels.reshape(-1, 1))
    # CREATE CONFUSION MATRIX
    #confusion = metrics.confusion_matrix(test_labels, predictions, labels=[0,1,2])
    
    # PULL ACCURACY FROM METRICS
    class_report = metrics.classification_report(test_labels, predict)
    roc_auc = metrics.roc_auc_score(test_labels, predict_proba, multi_class='ovo', average='macro')
    
    # RETURN THE CREATED SCORES
    return class_report, roc_auc

In [37]:
# C:\Users\Alex-\Desktop\Beslutsstödssystem\models-decision-system\MockPipeline
import pickle
def load_pickle(path):
    return pickle.load(open(path, 'rb'))

In [38]:
%matplotlib notebook
#new_data = load_pickle("cls-dataset.pickle")
x_test = pickle.load(open("test-dataset.pickle", 'rb'))

In [39]:
y_test = pd.read_csv("sick-dataset.csv")

In [40]:
y_test = y_test["label"]

In [41]:
x_test

array([[1.19671188, 1.17790008, 1.08383954],
       [1.20023295, 1.17430663, 1.1292845 ],
       [1.20894363, 1.17983079, 1.13130617],
       ...,
       [1.12289069, 1.10927844, 1.22247922],
       [1.12372512, 1.10002017, 1.19581151],
       [1.12451024, 1.08441639, 1.1518805 ]])

In [42]:
train_data = pd.read_csv("train-dataset.csv")

In [43]:
train_data

,Unnamed: 0,linreg_0,lstm_1,tcn_2,label
0,0,1.415416,1.416899,1.155509,2
1,1,1.411079,1.406109,1.231431,1
2,2,1.419329,1.404692,1.114336,1
3,3,1.433976,1.415452,1.147940,2
4,4,1.422778,1.413148,1.328912,1
...,...,...,...,...,...
2065,2065,1.195438,1.116277,1.085387,1
2066,2066,1.201720,1.120028,1.138446,2
2067,2067,1.202334,1.129956,1.153759,1
2068,2068,1.206246,1.134536,1.143560,0


In [44]:
train_data.columns[4]

'label'

In [45]:
x_train = train_data[train_data.columns[1:4]].to_numpy()

In [46]:
x_train

array([[1.41541576, 1.4168985 , 1.1555091 ],
       [1.41107941, 1.4061091 , 1.2314305 ],
       [1.41932869, 1.4046925 , 1.1143359 ],
       ...,
       [1.20233412, 1.1299561 , 1.1537595 ],
       [1.2062462 , 1.1345357 , 1.14356   ],
       [1.20233363, 1.1372457 , 1.1534529 ]])

In [47]:
y_train = train_data[train_data.columns[4]].to_numpy()

In [48]:
x

array([[0.981647  , 0.36637611, 0.14569776, 0.14285714, 0.47564914],
       [0.88785604, 0.28644626, 0.15063355, 0.14285714, 0.36914824],
       [0.90311048, 0.21378626, 0.13372386, 0.14285714, 0.43570668],
       ...,
       [0.16338935, 0.35569824, 0.40025716, 0.71428571, 0.57863884],
       [0.24693124, 0.55321847, 0.43293462, 0.85714286, 0.71704691],
       [0.25884877, 0.55783172, 0.471166  , 0.85714286, 0.71375575]])

# Random search grid

In [49]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

In [50]:
dataset = {"x_train": x_train, 
           "y_train": y_train,
           "x_test": x_test,
           "y_test": y_test,
           "features": x,
           "labels": y
          }

## Random forest

In [51]:
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    'grid_params': { # parameters for grid search (specific to each model)
        'n_estimators': [10, 50, 100, 500], # int, default=100
        'max_features': ['log2', 'sqrt'], # {“sqrt”, “log2”}, int or float, default=”auto”
        'max_depth': [10,100, None], # int, default=None
        'min_samples_split': [3, 10], # int or float, default=2
        'min_samples_leaf': [2, 4], # int or float, default=1
        'bootstrap': [True, False], # bool, default=True
        "class_weight": ["balanced"], # {“balanced”, “balanced_subsample”}, dict or list of dicts, default=None
        "random_state": [42]
    }
}

In [52]:
rfc = RandomForestClassifier()
rfc_model = grid_search_model(rfc, dataset, grid_settings)

Fitting 3 folds for each of 192 candidates, totalling 576 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 576 out of 576 | elapsed:   40.0s finished


In [53]:
rfc_model

RandomForestClassifier(class_weight='balanced', max_depth=100,
                       max_features='log2', min_samples_leaf=2,
                       min_samples_split=3, n_estimators=500, random_state=42)

In [96]:
rfc_model.fit(x_train, y_train)
class_repo, roc_auc = evaluate(rfc_model, x_test, y_test)
print(class_repo)
print(roc_auc)

              precision    recall  f1-score   support

           0       0.19      0.46      0.27       119
           1       0.19      0.13      0.16       114
           2       0.62      0.41      0.50       385

    accuracy                           0.37       618
   macro avg       0.34      0.33      0.31       618
weighted avg       0.46      0.37      0.39       618

0.4855959608723872


0 = buy
1 = sell
2 = hold

## Logistical Regression

In [92]:
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    'grid_params': {
        "C": [0.001, 0.1, 1, 10, 100],
        "penalty": ["l1", "l2", "elasticnet", "none"],
        "solver": ["lbfgs", "sag", "newton-cg"],
        "multi_class": ["auto", "multinomial", "ovo", "ovr"], # 
        "max_iter": [100,1000],
        "random_state": [0],
        "class_weight": ["balanced"]
    }
}


'grid_params': {
        "C": [0.001, 0.1, 1, 10, 100, 1000],
        "penalty": ["l1", "l2", "elasticnet", "none"],
        "solver": ["lbfgs", "sag", "newton-cg"],
        "multi_class": ["auto", "multinomial", "ovo", "ovr"], # 
        "max_iter": [100,1000, 10000],
        "random_state": [0],
        "class_weight": ["balanced"]
    }

In [93]:
logreg = LogisticRegression()
logreg_model = grid_search_model(logreg, dataset, grid_settings)

Fitting 3 folds for each of 480 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 900 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:    2.0s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


In [94]:
logreg_model

LogisticRegression(C=0.001, class_weight='balanced', multi_class='ovr',
                   penalty='none', random_state=0)

In [95]:
logreg_model.fit(x_train, y_train)
class_repo, roc_auc = evaluate(logreg_model, x_test, y_test)
print(class_repo)
print(roc_auc)

              precision    recall  f1-score   support

           0       0.19      0.60      0.28       119
           1       0.21      0.27      0.24       114
           2       0.54      0.13      0.21       385

    accuracy                           0.24       618
   macro avg       0.31      0.33      0.24       618
weighted avg       0.41      0.24      0.23       618

0.47078378400802107


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


## SVC

In [59]:
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{
        "C": [0.1, 1,10,100,1000],
        "kernel": ["linear", "rbf", "sigmoid"]
    }
}

In [60]:
svc = SVC(probability=True)
svc_model = grid_search_model(svc, dataset, grid_settings)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   28.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   28.5s finished


In [61]:
svc_model

SVC(C=0.1, kernel='linear', probability=True)

In [62]:
svc_model.fit(x_train, y_train)
class_repo, roc_auc = evaluate(svc_model, x_test, y_test)
print(class_repo)
print(roc_auc)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       119
           1       0.00      0.00      0.00       114
           2       0.62      1.00      0.77       385

    accuracy                           0.62       618
   macro avg       0.21      0.33      0.26       618
weighted avg       0.39      0.62      0.48       618

0.43101984015296707


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## GaussianNB

In [63]:
from sklearn.naive_bayes import GaussianNB
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{
        "var_smoothing": [1e-9, 1e-10, 1e-5]
    }
}

In [64]:
gnb = GaussianNB()
gnb_model = grid_search_model(gnb, dataset, grid_settings)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.0s finished


In [65]:
gnb_model

GaussianNB()

In [66]:
gnb_model.fit(x_train, y_train)
class_repo, roc_auc = evaluate(gnb_model, x_test, y_test)
print(class_repo)
print(roc_auc)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       119
           1       0.00      0.00      0.00       114
           2       0.62      1.00      0.77       385

    accuracy                           0.62       618
   macro avg       0.21      0.33      0.26       618
weighted avg       0.39      0.62      0.48       618

0.433910119071552


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Complemntary NB

In [67]:
from sklearn.naive_bayes import ComplementNB
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{

        "alpha": [0.5, 1],
        "norm": [True, False],
        
    }
}

In [68]:
cnb = ComplementNB()
cnb_model = grid_search_model(cnb, dataset, grid_settings)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.0s finished


In [69]:
cnb

ComplementNB()

In [70]:
cnb_model.fit(x_train, y_train)
class_repo, roc_auc = evaluate(cnb_model, x_test, y_test)
print(class_repo)
print(roc_auc)

              precision    recall  f1-score   support

           0       0.19      0.95      0.32       119
           1       0.33      0.03      0.05       114
           2       0.47      0.02      0.04       385

    accuracy                           0.20       618
   macro avg       0.33      0.33      0.14       618
weighted avg       0.39      0.20      0.09       618

0.4381860878322621


## 3-way confusion matrix

| | pred buy | pred sell | pred hold |
| --------- |:---------:|:---------:|----------:|        
| true buy  | | | |
| true sell | | | |
| true hold | | | |

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned | $1600 |
| col 2 is      | centered      |   $12 |
| zebra stripes | are neat      |    $1 |

### Create random search

In [71]:

rfc = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator=rfc,
                              param_distributions=random_grid,
                              n_iter=100,
                              cv = 3,
                              verbose=2,
                              random_state=42,
                              n_jobs=-1)

rf_random.fit(x_train_2, y_train_2)

NameError: name 'random_grid' is not defined

In [ ]:
base_model = RandomForestClassifier(random_state=4)
base_model.fit(x_train_2, y_train_2)
base_accuracy = evaluate(base_model, x_test_2, y_test_2)
print("==========================")
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, x_test_2, y_test_2)

In [ ]:
base_model.n_estimators

In [ ]:
rf_random.best_params_

#### Create gridsearch params from best values in random search

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 100, 110],
    'max_features': [2, 3, 'sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [2,3,4],
    'n_estimators': [1400, 1600, 2500]
}

In [ ]:
#n = np.linspace(0.8 * rf_random.best_params_["n_estimators"], 1.2 * rf_random.best_params_["n_estimators"], num=5)
#ms = np.linspace(0.7 * rf_random.best_params_["min_samples_split"], 1.3 * rf_random.best_params_["min_samples_split"], num=3)
#ml = np.linspace(0.8 * rf_random.best_params_["min_samples_leaf"], 1.2 * rf_random.best_params_["min_samples_leaf"], num=4)
#*md = np.linspace(0.8 * rf_random.best_params_["max_depth"], 1.2 * rf_random.best_params_["max_depth"], num=4)

In [ ]:
# Create the parameter grid based on the results of random search 
# Create a based model

rf = RandomForestClassifier()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_2, y_train_2)

In [ ]:
grid_search.fit(x_train_2, y_train_2)

grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, x_test_2, y_test_2)

In [ ]:
best_grid_model = best_grid

In [ ]:
best_grid_model.fit(x_train_2, y_train_2)
model_output = evaluate(best_grid_model, x_test_2, y_test_2)

In [ ]:
model_output

In [ ]:
bootstrap = grid_search.best_params_["bootstrap"]
max_depth = grid_search.best_params_["max_depth"]
max_features = grid_search.best_params_["max_features"]
min_samples_leaf = grid_search.best_params_["min_samples_leaf"]
min_samples_split = grid_search.best_params_["min_samples_split"]
n_estimators = grid_search.best_params_["n_estimators"]

In [ ]:
rfc = RandomForestClassifier(bootstrap=bootstrap, 
                             max_depth=max_depth, 
                             max_features=max_features, 
                             min_samples_leaf=min_samples_leaf,
                            min_samples_split=min_samples_split,
                            n_estimators=n_estimators)
rfc.fit(x_train_2, y_train_2)
model_output = evaluate(rfc, x_test_2, y_test_2)

In [ ]:
model_output[3]

In [ ]:
{'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 400}

## random forest

In [ ]:
y_train

In [ ]:


clf = RandomForestClassifier(max_depth=5, 
                             random_state=0, 
                             n_estimators=100,
                             class_weight="balanced"
                             #class_weight="[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 0}]"
                            )
clf.fit(x_train_2,y_train_2)
predict = clf.predict(x_test_2)

In [ ]:
predict

In [ ]:
def svc(x_train, y_train, x_test, y_test):
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 0.06, 'scale'],
                     'C': [1, 10, 15, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000, 10000]}]
    scores = ['precision', 'recall']
    
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(
            SVC(max_iter=10000, random_state=5, class_weight="balanced", decision_function_shape='ovo'), 
            tuned_parameters, 
            scoring='%s_macro' % score,
            cv = 5
        )
        clf.fit(x_train, y_train)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        y_true, y_pred = y_test, clf.predict(x_test)
        print(classification_report(y_true, y_pred))
        print(y_pred)
        return y_true, y_pred

In [ ]:
y_true, y_pred = svc(x_train, y_train_2, x_test, y_test_2)

In [ ]:
#data_eurusd = pd.read_csv("../EURUSD1m.csv")
data_mock = pd.read_csv("../MockData.csv")

In [ ]:
data_eurusd.head()

In [ ]:
data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [ ]:
os.chdir("..")
import Models